# Deep Learning with Python

## 6.1  Working with text data

> 处理文本数据

要用深度学习的神经网络处理文本数据，和图片类似，也要把数据向量化：文本 -> 数值张量。

要做这种事情可以把每个单词变成向量，也可以把字符变成向量，还可以把多个连续单词或字符(称为 *N-grams*)变成向量。

反正不管如何划分，我们把文本拆分出来的单元叫做 *tokens*（标记），拆分文本的过程叫做 *tokenization*(分词)。

> 注：token 的中文翻译是“标记”😂。这些翻译都怪怪的，虽然 token 确实有标记这个意思，但把这里的 token 翻译成标记就没内味儿了。我觉得 token 是那种以一个东西代表另一个东西来使用的意思，这种 token 是一种有实体的东西，比如代金券。“标记”这个词在字典上作名词是「起标示作用的记号」的意思，而我觉得记号不是个很实体的东西。代金券不是一种记号、也就能说是标记，同样的，这里的 token 也是一种实体的东西，我觉得不能把它说成是“标记”。我不赞同这种译法，所以下文所有涉及 token 的地方统一写成 “token”，不翻译成“标记”。


文本的向量化就是先作分词，然后把生成出来的 token 逐个与数值向量对应起来，最后拿对应的数值向量合成一个表达了原文本的张量。其中，比较有意思的是如何建立 token 和 数值向量 的联系，下面介绍两种搞这个的方法：one-hot encoding(one-hot编码) 和 token embedding(标记嵌入)，其中 token embedding 一般都用于单词，叫作词嵌入「word embedding」。

![文本的向量化：从文本到token再到张量](https://tva1.sinaimg.cn/large/007S8ZIlgy1ghek3mhp38j31320mg0v8.jpg)


### n-grams 和词袋(bag-of-words)

n-gram 是能从一个句子中提取出的 ≤N 个连续单词的集合。例如：「The cat sat on the mat.」

这个句子分解成 2-gram 是：

```
{"The", "The cat", "cat", "cat sat", "sat",
  "sat on", "on", "on the", "the", "the mat", "mat"}
```

这个集合被叫做 bag-of-2-grams (二元语法袋)。

分解成 3-gram 是：

```
{"The", "The cat", "cat", "cat sat", "The cat sat",
  "sat", "sat on", "on", "cat sat on", "on the", "the",
  "sat on the", "the mat", "mat", "on the mat"}
```

这个集合被叫做 bag-of-3-grams (三元语法袋)。

把这东西叫做「袋」是因为它只是 tokens 组成的集合，没有原来文本的顺序和意义。把文本分成这种袋的分词方法叫做「词袋(bag-of-words)」。

由于词袋是不保存顺序的（分出来是集合，不是序列），所以一般不在深度学习里面用。但在轻量级的浅层文本处理模型里面，n-gram 和词袋还是很重要的方法的。

### one-hot 编码

one-hot 是比较基本、常用的。其做法是将每个 token 与一个唯一整数索引关联， 然后将整数索引 i 转换为长度为 N 的二进制向量(N 是词表大小)，这个向量只有第 i 个元素为 1，其余元素都为 0。

下面给出玩具版本的 one-hot 编码示例：

In [1]:
# 单词级的 one-hot 编码

import numpy as np

samples = ['The cat sat on the mat.', 'The dog ate my homework.']

token_index = {}
for sample in samples:
    for word in sample.split():
        if word not in token_index:
            token_index[word] = len(token_index) + 1
            
# 对样本进行分词。只考虑每个样本前 max_length 个单词
max_length = 10

results = np.zeros(shape=(len(samples), 
                          max_length, 
                          max(token_index.values()) + 1))
for i, sample in enumerate(samples):
    for j, word in list(enumerate(sample.split()))[:max_length]:
        index = token_index.get(word)
        results[i, j, index] = 1.

print(results)

[[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]]


In [3]:
# 字符级的 one-hot 编码

import string

samples = ['The cat sat on the mat.', 'The dog ate my homework.']

characters = string.printable    # 所有可打印的 ASCII 字符
token_index = dict(zip(range(1, len(characters) + 1), characters))

max_length = 50
results = np.zeros((len(samples), max_length, max(token_index.keys()) + 1))
for i, sample in enumerate(samples):
    for j, character in enumerate(sample):
        index = token_index.get(character)
        results[i, j, index] = 1.
        
print(results)

[[[1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]


Keras 内置了比刚才写的这种玩具版本强大得多的 one-hot 编码工具：

In [7]:
from tensorflow.keras.preprocessing.text import Tokenizer

samples = ['The cat sat on the mat.', 'The dog ate my homework.']

tokenizer = Tokenizer(num_words=1000)    # 只考虑前 1000 个最常见的单词
tokenizer.fit_on_texts(samples)

sequences = tokenizer.texts_to_sequences(samples)    # 将字符串转换为整数索引组成的列表

one_hot_results = tokenizer.texts_to_matrix(samples, mode='binary')

word_index = tokenizer.word_index    # 单词索引，就是词表字典啦，用这个就可以还原数据

print(f'Found {len(word_index)} unique tokens.')
print(one_hot_results, word_index, sep='\n')

Found 9 unique tokens.
[[0. 1. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]
{'the': 1, 'cat': 2, 'sat': 3, 'on': 4, 'mat': 5, 'dog': 6, 'ate': 7, 'my': 8, 'homework': 9}
